## 필요한 라이브러리

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras import models
from keras import layers

## ImageDataGenerator로 데이터 배열 생성

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
## 훈련 데이터 증식
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=50,
    class_mode='binary')

val_generator = val_datagen.flow_from_directory(
    'dataset/val',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=20,
    class_mode='binary')

## Model

In [ ]:
from keras.applications import VGG16

conv_base = VGG16(weights = 'imagenet',
                 include_top=False,
                 input_shape=(IMG_SIZE, IMG_SIZE, 3))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
from keras import models
from keras import layers
from keras import optimizers
from keras import regularizers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())

model.add(layers.Dropout(0.5)) # 드롭아웃

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=1e-5),
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.summary()

## fit_generator로 모델 훈련하기

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=35,
    epochs=30,
    validation_data=val_generator,                                                     
    validation_steps=5)

## History 저장

In [ ]:
import pandas as pd

hist_df = pd.DataFrame(history.history)

hist_json_file = 'history_DenseNet_last_m4.json'
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)
    
hist_csv_file = 'history_DenseNet_last_m4.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

## evaluate_generator로 평가하기

In [ ]:
from keras.models import load_model
model = load_model('VGG16_Model1.h5')

model.evaluate_generator(test_generator, steps=10)

## predict_generator로 예측하기

In [ ]:
model.predict_generator(test_generator, steps=10)